# 第5章: 係り受け解析

> 日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルが[ai.ja.zip](https://nlp100.github.io/data/ai.ja.zip)に収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

In [ ]:
import os
from collections import defaultdict

import pydot
import japanize_matplotlib
from graphviz import Digraph
from IPython.display import Image,display_png
import matplotlib.pyplot as plt

In [ ]:
# データを保存するディレクトリの作成
DATADIR = "data"
CURRENTDIR = "/workspace/notebook"
CHAPDIR = os.path.join(DATADIR, "chapter5")

try:
    os.mkdir(CHAPDIR)
except:
    print("作成済み等の理由でディレクトリが作成されませんでした")

In [ ]:
filename = "ai.ja.txt.parsed"
in_fpath = os.path.join(CHAPDIR, filename)

In [ ]:
%cd $CURRENTDIR/$CHAPDIR/
!wget https://nlp100.github.io/data/ai.ja.zip
!unzip ai.ja.zip

In [ ]:
!cabocha -f1 -o ai.ja.txt.parsed ai.ja.txt 

In [ ]:
# 確認
!head -10 ./ai.ja.txt.parsed

In [ ]:
%cd $CURRENTDIR

## 40. 係り受け解析結果の読み込み（形態素）  

> 形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [ ]:
class Morph:
    def __init__(self, morph):
        # タブで二つに区切られている
        fields = morph.split("\t")
        self.surface = fields[0]
        # タブで区切られた2つ目の要素はカンマ区切り
        attr = fields[1].split(",")
        self.base = attr[6]  
        self.pos = attr[0]  
        self.pos1 = attr[1]  
        

In [ ]:
sentences = list()
morphs = list()

with open(in_fpath, "r", encoding="utf8")as fr:
    for line in fr:
        if (line != "EOS\n") and (line[0] != '*'):
            morphs.append(Morph(line))
        elif line == "EOS\n":
            sentences.append(morphs)
            morphs = []

for m in sentences[2]:
    print(vars(m))

## 41. 係り受け解析結果の読み込み（文節・係り受け）  

> 40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．


CaboChaの出力

```text
* 0 -1D 1/1 0.000000
人工	名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー
知能	名詞,一般,*,*,*,*,知能,チノウ,チノー
EOS
EOS
* 0 17D 1/1 0.388993
人工	名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー
知能	名詞,一般,*,*,*,*,知能,チノウ,チノー
* 1 17D 2/3 0.613549
（	記号,括弧開,*,*,*,*,（,（,（* 0 -1D 1/1 0.000000
人工	名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー
知能	名詞,一般,*,*,*,*,知能,チノウ,チノー
EOS
```

In [ ]:
class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs  # 形態素（Morphオブジェクト）のリスト
        self.dst = dst        # 係り先文節インデックス番号
        self.srcs = []        # 係り元文節インデックス番号のリスト 

In [ ]:
sentences = [] # 各文の情報を追加
chunks = [] # 各文節リスト
morphs = [] # 各文節の形態素リスト
src_idxs = defaultdict(list) # 「係り先idx: [係り元idxs（リスト）]」の辞書

with open(in_fpath, "r", encoding="utf8")as fr:    
    for line in fr:
        # 文節情報 or 形態素情報 or EOS で場合分け
        if line.startswith("*"):
            # 文節情報の時
            if morphs:
                chunks.append(Chunk(morphs, dst))
                morphs = [] # 次の文節なので初期化
            
            chunk_info = line.rstrip("\n").split(" ")
            chunk_idx = int(chunk_info[1])
            dst = int(chunk_info[2].rstrip("D"))
            if dst != -1:
                src_idxs[dst].append(chunk_idx)
        elif not line.startswith("EOS\n"):
            # 形態素情報の時
            morphs.append(Morph(line))
        else:
            # EOSのとき
            # 空行もあるので飛ばす
            if morphs:
                chunks.append(Chunk(morphs, dst))
                if src_idxs:
                    for chunk_idx, srcs in src_idxs.items():
                        chunks[chunk_idx].srcs = srcs
                if chunks:
                    sentences.append(chunks)
            # 初期化
            morphs = [] 
            chunks = []
            src_idxs = defaultdict(list)
    

In [ ]:
# 確認
for i, chunk in enumerate(sentences[1]):
    print(i, vars(chunk))

## 42. 係り元と係り先の文節の表示  

> 係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [ ]:
def get_pair(chunks):
    pairs = []
    for chunk in chunks:
        dst = chunk.dst
        if dst!= -1:
            # 係り元文節
            modifier = "".join([morph.surface if morph.pos != "記号" else "" for morph in chunk.morphs])
            # 係り先文節
            modifiee = "".join([morph.surface if morph.pos != "記号" else "" for morph in chunks[dst].morphs])
            pairs.append([modifier, modifiee])
    return pairs
    
for pair in get_pair(sentences[1]):
    print("\t".join(pair))

## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出  

> 名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [ ]:
def get_norn_verb_pair(chunks):
    pairs = []
    for chunk in chunks:
        dst = chunk.dst
        if dst!= -1:
            # 係り元文節
            # 名詞を含むかの確認
            if "名詞" in [morph.pos for morph in chunk.morphs]:
                modifier = "".join([morph.surface if morph.pos != "記号" else "" for morph in chunk.morphs])
                # 係り先文節
                # 動詞を含むかの確認
                if "動詞" in [morph.pos for morph in chunks[dst].morphs]:
                    modifiee = "".join([morph.surface if morph.pos != "記号" else "" for morph in chunks[dst].morphs])
                    pairs.append([modifier, modifiee])
    return pairs


for pair in get_norn_verb_pair(sentences[1]):
    print("\t".join(pair))

## 44. 係り受け木の可視化  

> 与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，Graphviz等を用いるとよい．

In [ ]:
def get_edges(chunks):
    edges = []
    for idx, chunk in enumerate(chunks):
        dst = chunk.dst
        if dst!= -1:
            modifier = "".join([morph.surface if morph.pos != "記号" else "" for morph in chunk.morphs])
            modifier_idx = f"({idx})"
            modifiee = "".join([morph.surface if morph.pos != "記号" else "" for morph in chunks[dst].morphs])
            modifiee_idx = f"({dst})"
            edges.append([f"{modifier}{modifier_idx}", f"{modifiee}{modifiee_idx}"])
    return edges

# グラフ保存ファイル
img_path = os.path.join(CHAPDIR, "ans44.png")

edges = get_edges(sentences[1])
g = pydot.graph_from_edges(edges, directed=True)
g.write_png(img_path)
display_png(Image(img_path))

## 45. 動詞の格パターンの抽出  

> 今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．
> * 動詞を含む文節において，最左の動詞の基本形を述語とする
> * 述語に係る助詞を格とする
> * 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
>
> 「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．
> >作り出す	で は を
>
> このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．
> * コーパス中で頻出する述語と格パターンの組み合わせ
> * 「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

In [ ]:
# 出力先
out_fpath = os.path.join(CHAPDIR, "ans45.txt")

In [ ]:
def get_verb_kaku(chunks):
    outputs = []
    for chunk in chunks:
        for morph in chunk.morphs:
            # 文節の左側の形態素から動詞チェック
            if morph.pos == "動詞":
                # 係り元の文節インデックスの確認
                kaku_patterns = []
                for src in chunk.srcs:
                    kaku_patterns.extend([morph.surface for morph in chunks[src].morphs if morph.pos == "助詞"])
                if kaku_patterns:
                    kaku_patterns = sorted(list(set(kaku_patterns)))
                    output_kaku_pattern = ' '.join(kaku_patterns)
                    outputs.append(f"{morph.base}\t{output_kaku_pattern}")
                    # 一番左の動詞とそれにかかる格助詞のみ取得するので
                    break
    return outputs

with open(out_fpath, "w", encoding="utf8") as fw:
    for sentence in sentences:
        if sentence:
            output_lines = get_verb_kaku(sentence)
            if output_lines:
                fw.write("\n".join(output_lines) + "\n")

In [ ]:
# 確認
!cat $out_fpath | sort | uniq -c | sort -nr | head -n 10

In [ ]:
!cat $out_fpath | grep '行う' | sort | uniq -c | sort -nr | head -n 5

In [ ]:
!cat $out_fpath | grep 'なる' | sort | uniq -c | sort -nr | head -n 5

## 46. 動詞の格フレーム情報の抽出

> 45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．
> * 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
> * 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる
> 
> 「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．
> 
> ```作り出す	で は を	会議で ジョンマッカーシーは 用語を```


### コメント

出力ルールが不明瞭。「助詞と同一の基準」の解釈が、  
「助詞と同じ順序に並び替えろ」なのか、「助詞と同じように辞書順に並び替えろ」なのかがわかりづらい。  
本回答では後者の解釈で実装している。

In [ ]:
# 出力先
out_fpath = os.path.join(CHAPDIR, "ans46.txt")

In [ ]:
def get_verb_kaku2(chunks):
    outputs = []
    for chunk in chunks:
        for morph in chunk.morphs:
            # 文節の左側の形態素から動詞チェック
            if morph.pos == "動詞":
                # 係り元の文節インデックスの確認
                kaku_patterns = []
                modifier = []
                for src in chunk.srcs:
                    kaku_patterns.extend([morph.surface for morph in chunks[src].morphs if morph.pos == "助詞"])
                    modifier.append("".join([morph.surface for morph in chunks[src].morphs if morph.pos != "記号"]))
                if kaku_patterns:
                    kaku_patterns = sorted(list(set(kaku_patterns)))
                    modifier = sorted(list(set(modifier)))
                    output_kaku_pattern = ' '.join(kaku_patterns)
                    output_modifier = ' '.join(modifier)
                    outputs.append(f"{morph.base}\t{output_kaku_pattern}\t{output_modifier}")
                    # 一番左の動詞とそれにかかる格助詞のみ取得するので
                    break
    return outputs

with open(out_fpath, "w", encoding="utf8") as fw:
    for sentence in sentences:
        if sentence:
            output_lines = get_verb_kaku2(sentence)
            if output_lines:
                fw.write("\n".join(output_lines) + "\n")

In [ ]:
# 確認
!cat $out_fpath| head -n 10

## 47. 機能動詞構文のマイニング

> 動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．
> * 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
> * 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
> * 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
> * 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）
> 
> 例えば「また、自らの経験を元に学習を行う強化学習という手法もある。」という文から，以下の出力が得られるはずである．
> >学習を行う	に を	元に 経験を

### コメント

- 述語にかかるという解釈で、述語が「サ変接続名詞+を+動詞の基本形」なのであれば、  それぞれの文節かかる文節すべてが必要になる
- 複数のサ変接続名詞が文節内に存在していることを考慮していない


**下の回答例は以下の特徴を持つ**

- 述語にかかる助詞と文節の並び順がおかしい
    - 係り元の文節に複数の助詞が含まれる場合、順序が崩れてしまう
- 複数のサ変接続名詞を持つ文節には対応している

In [ ]:
# 出力先
out_fpath = os.path.join(CHAPDIR, "ans47.txt")

In [ ]:
def get_verb_wokaku(chunks):
    outputs = []
    for chunk in chunks:
        for morph in chunk.morphs:
            # 文節の左側の形態素から動詞チェック
            if morph.pos == "動詞":
                # 係り元の文節インデックスの確認
                kaku_patterns = []
                modifier = []
                predicate = None
                for src in chunk.srcs:
                    if "サ変接続" in [morph.pos1 for morph in chunks[src].morphs] and \
                        "を" in [morph.surface for morph in chunks[src].morphs]:
                        predicate = "".join([morph.surface for morph in chunks[src].morphs if morph.pos1 == "サ変接続" or morph.surface == "を"])
                        predicate = f"{predicate}{morph.base}"
                    else:
                        kaku_patterns.extend([morph.surface for morph in chunks[src].morphs if morph.pos == "助詞"])
                        modifier.append("".join([morph.surface for morph in chunks[src].morphs if morph.pos != "記号"]))
                if predicate is not None:
                    output = f"{predicate}"
                    if kaku_patterns:
                        kaku_patterns = list(set(kaku_patterns))
                        modifier = list(set(modifier))
                        kaku_patterns, modifier = zip(*sorted(zip(kaku_patterns, modifier)))
                        output_kaku_pattern = ' '.join(kaku_patterns)
                        output_modifier = ' '.join(modifier)
                        output = f"{predicate}\t{output_kaku_pattern}\t{output_modifier}"
                    outputs.append(output)
                    # 一番左の動詞とそれにかかる格助詞のみ取得するので
                    break

    return outputs

with open(out_fpath, "w", encoding="utf8") as fw:
    for sentence in sentences:
        if sentence:
            output_lines = get_verb_wokaku(sentence)
            if output_lines:
                fw.write("\n".join(output_lines) + "\n")

In [ ]:
# 確認
!cat $out_fpath | cut -f 1 | sort | uniq -c | sort -nr | head -n 10

In [ ]:
!cat $out_fpath | cut -f 1,2 | sort | uniq -c | sort -nr | head -n 10

## 48. 名詞から根へのパスの抽出

> 文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．
> * 各文節は（表層形の）形態素列で表現する
> * パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する
> 
> 「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．
> >ジョンマッカーシーは -> 作り出した  
> >AIに関する -> 最初の -> 会議で -> 作り出した  
> >最初の -> 会議で -> 作り出した  
> >会議で -> 作り出した  
> >人工知能という -> 用語を -> 作り出した  
> >用語を -> 作り出した
> 
> KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．
> >ジョンマッカーシーは -> 作り出した  
> >ＡＩに -> 関する -> 会議で -> 作り出した  
> >会議で -> 作り出した  
> >人工知能と -> いう -> 用語を -> 作り出した  
> >用語を -> 作り出した
> 

In [ ]:
# 出力先
out_fpath = os.path.join(CHAPDIR, "ans48.txt")

In [ ]:
def get_norn_deps(chunks):
    outputs = []
    for chunk in chunks:
        # 名詞を含む文節なら
        if "名詞" in [morph.pos for morph in chunk.morphs]:
            path_chunks = [''.join([morph.surface for morph in chunk.morphs if morph.pos != "記号"])]
            # そこから係り先をすべて見ていく
            dst_chunk_idx = chunk.dst
            while dst_chunk_idx != -1:
                path_chunks.append("".join([morph.surface for morph in chunks[dst_chunk_idx].morphs if morph.pos != "記号"]))
                dst_chunk_idx = chunks[dst_chunk_idx].dst
            if len(path_chunks) > 1:
                outputs.append(" -> ".join(path_chunks))
    return outputs
    

with open(out_fpath, "w", encoding="utf8") as fw:
    for sentence in sentences:
        if sentence:
            output_lines = get_norn_deps(sentence)
            if output_lines:
                fw.write("\n".join(output_lines) + "\n")

## 49. 名詞間の係り受けパスの抽出

> 文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がiとj（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．
> * 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する
> * 文節iとjに含まれる名詞句はそれぞれ，XとYに置換する
> 
> また，係り受けパスの形状は，以下の2通りが考えられる．
> * 文節iから構文木の根に至る経路上に文節jが存在する場合: 文節iから文節jのパスを表示
> * 上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合: 文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を” | “で連結して表示
> 
> 「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．
> >Xは | Yに関する -> 最初の -> 会議で | 作り出した  
> >Xは | Yの -> 会議で | 作り出した  
> >Xは | Yで | 作り出した  
> >Xは | Yという -> 用語を | 作り出した  
> >Xは | Yを | 作り出した  
> >Xに関する -> Yの  
> >Xに関する -> 最初の -> Yで  
> >Xに関する -> 最初の -> 会議で | Yという -> 用語を | 作り出した  
> >Xに関する -> 最初の -> 会議で | Yを | 作り出した  
> >Xの -> Yで  
> >Xの -> 会議で | Yという -> 用語を | 作り出した  
> >Xの -> 会議で | Yを | 作り出した  
> >Xで | Yという -> 用語を | 作り出した  
> >Xで | Yを | 作り出した  
> >Xという -> Yを
> 
> KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．
> >Xは | Yに -> 関する -> 会議で | 作り出した。  
> >Xは | Yで | 作り出した。  
> >Xは | Yと -> いう -> 用語を | 作り出した。  
> >Xは | Yを | 作り出した。  
> >Xに -> 関する -> Yで  
> >Xに -> 関する -> 会議で | Yと -> いう -> 用語を | 作り出した。  
> >Xに -> 関する -> 会議で | Yを | 作り出した。  
> >Xで | Yと -> いう -> 用語を | 作り出した。  
> >Xで | Yを | 作り出した。  
> >Xと -> いう -> Yを
> 

### コメント

- いまいちルールがわかっていないのでパス